In [1]:
import pandas as pd

df = pd.read_csv('../dist/h.csv')
df_sw = pd.read_csv('./stopwords-id.csv')


In [60]:
from mpstemmer import MPStemmer
import string

stemmer = MPStemmer()
stemmer.check_nonstandard = False
stem = stemmer.stem_kalimat

translation = str.maketrans('', '', string.punctuation)
sw = list(df_sw['stopword'])
def preprocess(s):
    clean = s.translate(translation)
    stemmed = stem(clean)
    stripped = stemmed.strip()
    tokenized = stripped.split(' ')
    return tokenized

df['preprocessed'] = df['headline'].apply(preprocess)

In [61]:
from gensim.models.phrases import Phrases, Phraser

phrases2 = Phrases(df['preprocessed'], min_count=1, threshold=10)
phrases3 = Phrases(phrases2[df['preprocessed']], threshold=100)

bigram = Phraser(phrases2)
trigram = Phraser(phrases3)

df['preprocessed'] = list(trigram[df['preprocessed']])

In [62]:
def remove_numeric_token(token):
    return [w for w in token if w not in sw and not w.isnumeric()]

df['preprocessed'] = df['preprocessed'].apply(remove_numeric_token)

In [86]:
df.to_parquet('../dist/hp.parquet')